## Creating Interactive Visualizations using Bokeh - Homework 1

Author: Kriti Shrivastava (Spire ID: 31041848)

Datasource: http://www.gapminder.org

#### Description: 
Total 25-54 unemployed (% age 25-54 population): Percentage of total population, age group 25-54, that has been registered as unemployed during the given year(1981-2005).

#### Source Organization:
International Labour Organization
http://www.ilo.org/global/lang--en/index.htm

Key Indicators of the Labour Market Programme
http://www.ilo.org/public/english/employment/strat/kilm/download.htm



###### Bokeh Version used: 0.12.6

    Importing necessary packages

In [45]:
from bokeh.plotting import figure, output_notebook, show
import pandas
import os
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.layouts import row
from bokeh.layouts import column
from bokeh.models import HoverTool, CustomJS
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Select

    Reading CSV data to Dataframes

In [46]:
path = os.getcwd()
path = path + '\data\indicator_t 25-54 unemploy.csv'

data = pandas.read_csv(path)
source = ColumnDataSource(data)

path = os.getcwd()
path = path + '\data\mean_unemployment.csv'

avg_data = pandas.read_csv(path)
avg_source = ColumnDataSource(avg_data)

path = os.getcwd()
path = path + '\data\\unemployment_per_country.csv'

all_country_data = pandas.read_csv(path)
all_country_source = ColumnDataSource(all_country_data)

output_notebook()

Loading BokehJS ...

    Comparison between unemployment rates in 1981 and 2005.
    Interactions: Hover, Linked Brushing, Reset

In [47]:
Countries = list()
for country in data['Country']:
    if str(country) != 'nan':
        Countries.append(str(country))

# Scatterplot 1: unemployment rates in 1981
p = figure(plot_width=500, plot_height=500, y_range=Countries,
           tools="hover,box_select,lasso_select,help,reset", title="Unemployment % across the world in 2000")

p.circle( y='Country', x='2000', source=source, alpha=0.8, size=10)
p.xaxis.axis_label = 'Country'
p.yaxis.axis_label = 'Unemployment %'

# Scatterplot 2: umemployment rates in 2005
p1 = figure(plot_width=500, plot_height=500, y_range=Countries,
           tools="hover,box_select,lasso_select,help,reset", title="Unemployment % across the world in 2005")

p1.circle( y='Country', x='2005', source=source, alpha=0.8, size=10)
p1.xaxis.axis_label = 'Country'
p1.yaxis.axis_label = 'Unemployment %'

show(row(p,p1))

    Trend in unemployment over the world between 1981-2005.
    Interactions: Custom Hover

In [48]:
# Line chart: mean unemployment rates during 1981-2005
hover = HoverTool(
        tooltips=[
            ("Year", "@Year"),
            ("Mean Unemployment %", "@Average"),
        ]
    )

print()
p = figure(plot_width=650, plot_height=500, tools=[hover], title="Mean Unemployment % across the world: 1981-2005")
p.line(x='Year', y='Average', line_width=1, alpha=0.5, source=avg_source)

p.xaxis.axis_label = 'Year'
p.yaxis.axis_label = 'Mean Unemployment %'
show(p)

    Unemployment trend in a country over 1981-2005.
    Interactions: Single selection widget (Custom JS)

In [49]:
country_data = {
    'Country': list(all_country_data['United States']),
    'Year': list(all_country_data['Year'])
}    
country_source = ColumnDataSource(data=country_data)

ColumnDataSource(country_data['Year'], country_data['Country'])

callback =  CustomJS(args=dict(country_source=country_source, all_country_source=all_country_source), code="""
        var data = country_source.data;  
        var all_data = all_country_source.data;
        var selected_country = cb_obj.value
        data['Country'] = all_data[selected_country]
        x = data['Year']
        y = data['Country']
        console.log(country_source)
        country_source.change.emit();
    """)
    
plot = figure(plot_width=400, plot_height=400)
plot.line('Year', 'Country', source=country_source, line_width=3, line_alpha=0.6)

select = Select(title="Select Country:", value="United States", options=Countries, callback=callback)

layout = column(select, plot)

show(layout)